<a href="https://colab.research.google.com/github/Bhoomika685/Manager-summarization-task1/blob/main/Manager_Task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# ============================
# Step 1: Install dependencies
# ============================
!pip install transformers --quiet

from transformers import pipeline

# Load summarizer model
summarizer = pipeline("summarization", model="t5-small", tokenizer="t5-small")

# ============================
# Step 2: Define Conversation Manager
# ============================
class ConversationManager:
    def __init__(self, trunc_limit_turns=None, trunc_limit_words=None, k_periodic=3):
        self.history = []  # full conversation history
        self.k_periodic = k_periodic  # summarize after every k turns
        self.trunc_limit_turns = trunc_limit_turns
        self.trunc_limit_words = trunc_limit_words
        self.summaries = []  # store summaries

    def add_message(self, speaker, message):
        """Add user/assistant message and check truncation + summarization."""
        self.history.append(f"{speaker}: {message}")
        self.apply_truncation()
        if len(self.history) % self.k_periodic == 0:
            self.perform_summarization()

    def apply_truncation(self):
        """Apply truncation by turns or word length."""
        # Limit by turns
        if self.trunc_limit_turns is not None:
            self.history = self.history[-self.trunc_limit_turns:]
        # Limit by word count
        if self.trunc_limit_words is not None:
            words = " ".join(self.history).split()
            if len(words) > self.trunc_limit_words:
                words = words[-self.trunc_limit_words:]
                self.history = [" ".join(words)]

    def perform_summarization(self):
        """Summarize conversation so far and replace history with summary."""
        text = " ".join(self.history)
        summary = summarizer(text, max_length=60, min_length=10, do_sample=False)[0]['summary_text']
        self.summaries.append(summary)
        self.history = [f"Summary so far: {summary}"]

    def show_history(self):
        """Print conversation history."""
        print("\n--- Conversation History ---")
        for msg in self.history:
            print(msg)

    def show_summaries(self):
        """Print stored summaries."""
        print("\n--- Stored Summaries ---")
        for i, s in enumerate(self.summaries, 1):
            print(f"Summary {i}: {s}")


# ============================
# Step 3: DEMO
# ============================
manager = ConversationManager(trunc_limit_turns=5, trunc_limit_words=50, k_periodic=3)

# Simulate conversation
manager.add_message("User", "Hello, I need help with Python loops.")
manager.add_message("Assistant", "Sure, I can explain Python loops like for and while.")
manager.add_message("User", "Great! Can you also show me examples with code?")
manager.show_history()

# Next 3 turns
manager.add_message("Assistant", "Of course, here’s a simple for loop that prints numbers 1 to 5.")
manager.add_message("User", "Thanks, what about while loop?")
manager.add_message("Assistant", "A while loop runs until a condition becomes false.")
manager.show_history()

# Check summaries
manager.show_summaries()

Device set to use cpu
Your max_length is set to 60, but your input_length is only 43. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=21)
Both `max_new_tokens` (=256) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Your max_length is set to 60, but your input_length is only 51. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=25)
Both `max_new_tokens` (=256) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/t


--- Conversation History ---
Summary so far: Assistant: Sure, I can explain Python loops like for and while .

--- Conversation History ---
Summary so far: Assistant: I can explain Python loops like for and while loops .
Assistant: A while loop runs until a condition becomes false.

--- Stored Summaries ---
Summary 1: Assistant: Sure, I can explain Python loops like for and while .
Summary 2: Assistant: I can explain Python loops like for and while loops .
